In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-08"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14250,2024-03-08,Eua Nba,00:00,Denver Nuggets,Boston Celtics,2.03,1.84,220.5,1.83,1.95,1.5,1.81,1.69,Eg54JGFj,0.492611,0.543478,0.546448,0.512821,0.036089,0.2,0.4,NaN,NaN,187.188,36.160977,0.193180,154.352,10.703120,0.069342,148.032,175.518,134.26,168.99,0.0,0.0,0.0,0.0,0.069432,0.044896,0.048487,0.33,0.066,15.606061,0.802596,0.8,-0.002596,2.11,0.422,1.990521,0.672526,0.8,0.127474
14251,2024-03-08,Eua Nba,00:00,Golden State Warriors,Chicago Bulls,1.29,3.82,222.5,1.86,1.96,-8.5,1.97,7.50,Gbqmr0hi,0.775194,0.261780,0.537634,0.510204,0.036974,0.0,0.8,NaN,NaN,198.170,55.735046,0.281249,338.080,99.755713,0.295065,183.910,295.224,207.50,235.62,0.0,0.0,0.0,0.0,0.700188,0.037021,0.825829,1.23,0.246,1.178862,0.622255,0.6,-0.022255,0.73,0.146,19.315068,0.455566,0.5,0.044434
14252,2024-03-08,Eua Nba,00:00,Sacramento Kings,San Antonio Spurs,1.18,5.28,235.5,1.93,1.93,-10.5,1.86,11.00,U1misK7c,0.847458,0.189394,0.518135,0.518135,0.036852,0.8,0.4,NaN,NaN,238.510,82.814147,0.347215,489.950,136.213947,0.278016,167.112,489.950,147.40,602.68,0.0,1.0,0.0,0.0,0.897566,0.000000,1.005125,-0.69,-0.138,-1.304348,0.738008,0.5,-0.238008,-1.32,-0.264,-16.212121,0.284015,0.2,-0.084015
14253,2024-03-08,Eua Nba,21:00,Washington Wizards,Charlotte Hornets,1.71,1.98,226.5,1.85,1.95,-3.5,1.98,2.65,2FoaubxA,0.584795,0.505051,0.540541,0.512821,0.089846,0.2,0.6,NaN,NaN,667.258,271.017403,0.406166,455.296,146.758410,0.322336,497.296,747.428,276.85,570.36,0.0,1.0,0.0,0.0,0.103479,0.037216,0.204649,-5.00,-1.000,-0.710000,0.276984,0.0,-0.276984,6.90,1.380,0.710145,0.186065,0.2,0.013935
14254,2024-03-08,Eua Nba,21:30,Cleveland Cavaliers,Minnesota Timberwolves,1.93,1.93,208.5,1.83,1.98,-1.5,2.03,2.00,Kjt4vIiG,0.518135,0.518135,0.546448,0.505051,0.036269,0.6,0.6,NaN,NaN,170.350,22.877619,0.134298,170.294,15.832340,0.092971,162.962,218.522,145.04,196.62,0.0,0.0,0.0,0.0,0.000000,0.055678,0.010528,-0.78,-0.156,-5.961538,0.652128,0.8,0.147872,3.94,0.788,1.180203,0.649465,0.8,0.150535
14255,2024-03-08,Eua Nba,21:30,New York Knicks,Orlando Magic,1.74,1.93,207.5,1.83,1.97,-2.5,1.97,2.11,bsu8wx7M,0.574713,0.518135,0.546448,0.507614,0.092847,0.2,0.4,NaN,NaN,214.378,38.328711,0.178790,183.040,25.913201,0.141571,183.654,204.358,219.88,187.68,0.0,0.0,0.0,0.0,0.073215,0.052103,0.048527,-2.31,-0.462,-1.601732,0.658184,0.6,-0.058184,0.70,0.140,6.642857,0.465836,0.4,-0.065836
14256,2024-03-08,Eua Nba,22:00,Memphis Grizzlies,Atlanta Hawks,2.16,1.59,213.5,1.81,2.00,2.5,1.98,1.50,h4vCxdMS,0.462963,0.628931,0.552486,0.500000,0.091894,0.4,0.4,NaN,NaN,282.052,200.840976,0.712071,198.710,40.528811,0.203960,282.052,286.102,172.00,222.36,0.0,0.0,0.0,0.0,0.214960,0.070525,0.195064,3.04,0.608,1.907895,0.389395,0.3,-0.089395,-3.49,-0.698,-0.845272,0.462552,0.3,-0.162552
14257,2024-03-08,Europa Euroliga,16:30,Milano,Partizan,1.78,2.20,155.5,1.85,2.09,-2.5,2.01,2.55,vgr0RQ9c,0.561798,0.454545,0.540541,0.478469,0.016343,0.2,0.2,NaN,NaN,171.658,42.818548,0.249441,132.608,56.645516,0.427165,138.928,168.008,221.94,111.72,0.0,0.0,0.0,0.0,0.149239,0.086145,0.167473,2.59,0.518,1.505792,0.620638,0.7,0.079362,-2.73,-0.546,-2.197802,0.575264,0.3,-0.275264
14258,2024-03-08,Arábia Saudita Premier League,13:00,Al Wehda,Al Ittihad,1.87,1.85,168.5,1.80,1.88,-1.5,1.93,1.99,h4YHd49H,0.534759,0.540541,0.555556,0.531915,0.075300,0.6,0.8,NaN,NaN,182.930,74.842215,0.409130,111.290,18.729215,0.168292,132.024,132.066,105.00,110.40,0.0,0.0,0.0,0.0,0.00

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Chicago Bulls,222.5,1.86,1.0000,Over
1,00:00,Eua Nba,Sacramento Kings,San Antonio Spurs,235.5,1.93,1.0000,Over
2,21:00,Eua Nba,Washington Wizards,Charlotte Hornets,226.5,1.85,1.0000,Over
3,21:30,Eua Nba,Cleveland Cavaliers,Minnesota Timberwolves,208.5,1.83,1.0000,Over
4,21:30,Eua Nba,New York Knicks,Orlando Magic,207.5,1.83,1.0000,Over
5,16:30,Europa Euroliga,Milano,Partizan,155.5,1.85,1.0000,Over
6,07:30,Austrália Nbl,Perth Wildcats,Tasmania JackJumpers,179.5,1.80,0.9922,Over
7,16:45,Espanha Leb Ouro,Caceres,Estela,159.5,1.86,1.0000,Over
8,00:00,Eua Ncaa,Long Beach State,UC Santa Barbara,153.5,1.80,1.0000,Over
9,00:00,Eua Ncaa,Morehead State,Siu Edwardsville,135.5,1.80,1.0000,Over
